<a href="https://colab.research.google.com/github/Jvxoq/Fradulent-Transaction-Detection/blob/main/Fraud_Transaction_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import xgboost as xgb


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Project/fraudTrain.csv")

In [ ]:
df.head(5)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [ ]:
df.drop(columns=['Unnamed: 0', 'cc_num', 'trans_date_trans_time', 'merchant', 'first', 'last', 'street', 'city', 'state', 'zip', 'dob', 'trans_num', 'lat', 'long', 'job', 'merch_lat', 'merch_long'], inplace=True)

In [ ]:
df.head(5)

,category,amt,gender,city_pop,unix_time,is_fraud
0,misc_net,4.97,F,3495,1325376018,0
1,grocery_pos,107.23,F,149,1325376044,0
2,entertainment,220.11,M,4154,1325376051,0
3,gas_transport,45.00,M,1939,1325376076,0
4,misc_pos,41.96,M,99,1325376186,0


In [ ]:
print(df.columns)

Index(['category', 'amt', 'gender', 'city_pop', 'unix_time', 'is_fraud'], dtype='object')


In [ ]:
num_cols = ["amt", "city_pop", "unix_time"]

In [ ]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
df.head(5)

,category,amt,gender,city_pop,unix_time,is_fraud
0,misc_net,-0.407826,F,-0.282589,-1.858664,0
1,grocery_pos,0.230039,F,-0.293670,-1.858662,0
2,entertainment,0.934149,M,-0.280406,-1.858662,0
3,gas_transport,-0.158132,M,-0.287742,-1.858660,0
4,misc_pos,-0.177094,M,-0.293835,-1.858651,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
label_encoders = {}
for col in ["category", "gender"]:  # Add more if needed
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Convert categorical to numerical
    label_encoders[col] = le  # Store encoders for future decoding


In [ ]:
print(df[num_cols])

              amt  city_pop  unix_time
0       -0.407826 -0.282589  -1.858664
1        0.230039 -0.293670  -1.858662
2        0.934149 -0.280406  -1.858662
3       -0.158132 -0.287742  -1.858660
4       -0.177094 -0.293835  -1.858651
...           ...       ...        ...
1296670 -0.341769 -0.293309   1.757855
1296671 -0.116339 -0.293832   1.757855
1296672  0.221930 -0.291186   1.757856
1296673  0.028375 -0.290434   1.757861
1296674 -0.412005 -0.293441   1.757862

[1296675 rows x 3 columns]


In [ ]:
df.head(5)

,category,amt,gender,city_pop,unix_time,is_fraud
0,8,-0.407826,0,-0.282589,-1.858664,0
1,4,0.230039,0,-0.293670,-1.858662,0
2,0,0.934149,1,-0.280406,-1.858662,0
3,2,-0.158132,1,-0.287742,-1.858660,0
4,9,-0.177094,1,-0.293835,-1.858651,0


In [ ]:
X = df.drop(columns=['is_fraud'])

In [ ]:
y = df['is_fraud']

In [ ]:
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
df["anomaly_score"] = iso_forest.fit_predict(X)

In [ ]:
anomaly_indices = df[df["anomaly_score"] == -1].index

In [ ]:
df.head(5)

,category,amt,gender,city_pop,unix_time,is_fraud,anomaly_score
0,8,-0.407826,0,-0.282589,-1.858664,0,1
1,4,0.230039,0,-0.293670,-1.858662,0,1
2,0,0.934149,1,-0.280406,-1.858662,0,1
3,2,-0.158132,1,-0.287742,-1.858660,0,1
4,9,-0.177094,1,-0.293835,-1.858651,0,1


In [ ]:
X_resampled, y_resampled = SMOTE(sampling_strategy=0.3, random_state=42).fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [ ]:
xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, scale_pos_weight=10, random_state=42)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

In [ ]:
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"AUC-ROC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96    257613
           1       0.81      0.99      0.89     77571

    accuracy                           0.95    335184
   macro avg       0.91      0.96      0.93    335184
weighted avg       0.95      0.95      0.95    335184

AUC-ROC Score: 0.9957


In [ ]:
import pickle

# Assume 'model' is your trained ML model
with open("fraud_model.pkl", "wb") as file:
    pickle.dump(xgb_model, file)

print("Model saved as fraud_model.pkl")


Model saved as fraud_model.pkl


In [ ]:

# from sklearn.preprocessing import StandardScaler

# Save label encoders dynamically
with open("label_encoders.pkl", "wb") as file:
    pickle.dump(label_encoders, file)  # Saves all label encoders in a single file

print("Label encoders saved ")

# Save StandardScaler
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

print("Scaler saved ")



Label encoders saved 
Scaler saved 


In [ ]:
pip install gradio

In [ ]:
import gradio as gr
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the trained XGBoost model
with open("fraud_model.pkl", "rb") as file:
    model = pickle.load(file)

# Load the saved label encoders
with open("label_encoders.pkl", "rb") as file:
    label_encoders = pickle.load(file)

# Load the saved StandardScaler
with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

# Define numerical and categorical columns
num_cols = ["amt", "city_pop", "unix_time"]
cat_cols = ["category", "gender"]

# Function to handle unseen labels safely
def safe_label_encoding(label, encoder):
    if label in encoder.classes_:
        return encoder.transform([label])[0]
    else:
        print(f"⚠ Warning: '{label}' is an unseen category. Assigning default value.")
        return encoder.transform([encoder.classes_[0]])[0]  # Assigning the first known class

# Function to preprocess input data and predict fraud (CSV upload)
def predict_from_csv(file):
    try:
        df = pd.read_csv(file.name)

        # Ensure required columns exist
        missing_cols = [col for col in num_cols + cat_cols if col not in df.columns]
        if missing_cols:
            return f"Missing columns: {missing_cols}"

        # Apply label encoding safely
        for col in cat_cols:
            if col in label_encoders:
                df[col] = df[col].apply(lambda x: safe_label_encoding(x, label_encoders[col]))
            else:
                return f"Error: Label encoder for {col} not found."

        # Standardize numerical features
        df[num_cols] = scaler.transform(df[num_cols])

        # Drop target variable if present
        if "is_fraud" in df.columns:
            df.drop(columns=["is_fraud"], inplace=True)

        # Make predictions
        df["Fraud Prediction"] = model.predict(df)

        return df  # Return DataFrame with predictions

    except Exception as e:
        return f"Error: {e}"

# Function to predict fraud from manual inputs
def predict_from_inputs(category, amt, gender, city_pop, unix_time):
    try:
        # Encode categorical values safely
        category_encoded = safe_label_encoding(category, label_encoders["category"])
        gender_encoded = safe_label_encoding(gender, label_encoders["gender"])

        # Standardize numerical values
        scaled_features = scaler.transform([[amt, city_pop, unix_time]])

        # Create final input array
        final_input = np.hstack((category_encoded, scaled_features[0], gender_encoded)).reshape(1, -1)

        # Predict fraud
        prediction = model.predict(final_input)[0]

        return "Fraud" if prediction == 1 else "Not Fraud"

    except Exception as e:
        return f"Error: {e}"

# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 🚀 Fraud Detection Model")


    with gr.Tab("🔹 Test Single Transaction"):
        category = gr.Dropdown(["travel", "shopping", "food", "entertainment"], label="Category")
        amt = gr.Number(label="Transaction Amount ($)")
        gender = gr.Dropdown(["M", "F"], label="Gender")  # Ensure labels match training data
        city_pop = gr.Number(label="City Population")
        unix_time = gr.Number(label="Unix Timestamp")
        result = gr.Textbox(label="Prediction")
        predict_button = gr.Button("Predict")
        predict_button.click(predict_from_inputs, inputs=[category, amt, gender, city_pop, unix_time], outputs=result)

# Launch the Gradio App
app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f463d44110a5ba368a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
